In [1]:
from scipy.stats import norm
import numpy as np

In [2]:
S = 100
E = 100
T = 1
r = 0.05
sigma = 0.2
mu = 0.1

In [3]:
def black_scholes(S, K, T, r, sigma, type="call", t=0):
    d1 = (np.log(S / K) + (r + (sigma ** 2) / 2) * (T - t)) / (sigma * np.sqrt(T - t))
    d2 = d1 - sigma * np.sqrt(T - t)

    if type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)
    if type == "put":
        return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)

In [4]:
def binomial_method(S, E, r, sigma, T, market = "eu", option_type="call", steps = 100):
    R = np.exp(r * (T / steps))
    Rinv = 1.0 / R

    # розраховуємо u, d, p
    u = np.exp(sigma * np.sqrt(T / steps))
    d = 1.0 / u
    p_up = (R - d) / (u - d)
    p_down = 1.0 - p_up

    # розраховуємо ціни активу на кожному кроці
    prices = np.zeros(steps + 1)
    prices[0] = S * (d ** steps)
    for i in range(1, steps + 1):
        prices[i] = prices[i - 1] * (u * u)
    
    if option_type == "call":
        values = np.maximum(0, prices - E)
    else:
        values = np.maximum(0, E - prices)

    # рекурентно обчилюємо значення V_0
    if market == "eu":
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                values[i] = (p_up * values[i + 1] + p_down * values[i]) * Rinv
    
    if market == "us":
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                values[i] = (p_up * values[i + 1] + p_down * values[i]) * Rinv
                prices[i] = d * prices[i + 1]
                if option_type == "call":
                    values[i] = max(values[i], prices[i] - E)
                else:
                    values[i] = max(values[i], E - prices[i])
    
    return values[0]


In [5]:
european_call = binomial_method(S, E, r, sigma, T, market = "eu", option_type="call")
european_put = binomial_method(S, E, r, sigma, T, market = "eu", option_type="put")

In [6]:
call_price = black_scholes(S, E, T, r, sigma, 'call')
put_price = black_scholes(S, E, T, r, sigma, 'put')

In [7]:
call_price, put_price

(10.450583572185565, 5.573526022256971)

In [8]:
european_call, european_put

(10.430611662249516, 5.553554112321299)

In [9]:
american_call = binomial_method(S, E, r, sigma, T, market = "us", option_type="call")
american_put = binomial_method(S, E, r, sigma, T, market = "us", option_type="put")

In [10]:
american_call, american_put

(10.430611662249516, 6.082354409142225)